# network


In [ ]:
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
def dtmmaker(text):
    custom_tokenizer = CustomTokenizer()
    vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
    dtm = vectorizer.fit_transform(text)
    dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
    return dtm_df


In [ ]:
def adjmaker(dtm):
    co_occurrence_matrix = np.dot(dtm.T, dtm)
    binary_matrix = (co_occurrence_matrix > 0).astype(int)

    dtm_ad = np.dot(binary_matrix.T, binary_matrix)


    node_names = dtm.columns.tolist()
    dtm_matrix = pd.DataFrame(dtm_ad, index=node_names, columns=node_names)

    # Create a list of edges
    dtm_edges = []
    for i in range(len(dtm_ad)):
        for j in range(i, len(dtm_ad)):
            if dtm_ad[i][j] > 0:
                dtm_edges.append((node_names[i], node_names[j], dtm_ad[i][j]))

    # Create a dictionary from the edge tuples
    edge_dict = {'source': [edge[0] for edge in dtm_edges],
                'target': [edge[1] for edge in dtm_edges],
                'weight': [edge[2] for edge in dtm_edges]}

    # Create a dataframe from the dictionary and rename the index, columns, and values
    dtm_edges_df = pd.DataFrame.from_dict(edge_dict)

    dtm_edges_df = dtm_edges_df.sort_values('weight', ascending=False)

    dtm_edges_df_top = dtm_edges_df[:10000]

    return dtm_matrix, dtm_edges, dtm_edges_df_top


#pagerank


In [ ]:
def pagerankmerge(df):
  # Extract the first two columns as 2-tuples
  df_tuple = [tuple(row) for row in df.iloc[:, :2].values]
  G = nx.DiGraph()
  G.add_edges_from(df_tuple)

  # Calculate PageRank
  pagerank = nx.pagerank(G)

  # Convert PageRank dictionary to DataFrame
  pagerank_df = pd.DataFrame.from_dict(pagerank, orient='index', columns=['PageRank'])

  dff= df.iloc[:, :2]

  # Merge the edge list DataFrame with the PageRank DataFrame
  merged_df = pd.merge(dff, pagerank_df, left_on='source', right_index=True, how='left')
  return merged_df

In [ ]:
def pagerank(df):
  df_tuple = [tuple(row) for row in df.iloc[:, :2].values]
  G = nx.DiGraph()
  G.add_edges_from(df_tuple)
  pagerank = nx.pagerank(G)
  pagerank_df = pd.DataFrame.from_dict(pagerank, orient='index', columns=['PageRank'])
  return pagerank_df